In [3]:
# !pip install --quiet "torch==2.1.2" tensorboard

# # Install Hugging Face libraries
# !pip install --quiet --upgrade \
#   "sentence-transformers>=3" \
#   "datasets==2.19.1"  \
#   "transformers==4.41.2"

In [1]:
import json

from torch.utils.data import DataLoader
from sentence_transformers import InputExample
from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
import torch
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from sentence_transformers.util import cos_sim

/opt/conda/envs/env_jason/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
TRAIN_DATASET_FPATH = './generated_data/train_dataset.json'
VAL_DATASET_FPATH = './generated_data/val_dataset.json'

In [3]:
with open(TRAIN_DATASET_FPATH, 'r') as f:
    train_dataset = json.load(f)

with open(VAL_DATASET_FPATH, 'r') as f:
    val_dataset = json.load(f)

In [4]:
model_id = 'all-mpnet-base-v2'  # Hugging Face model ID

model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

/opt/conda/envs/env_jason/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# dataset = train_dataset

# BATCH_SIZE = 128

matryoshka_dimensions = [768, 512, 256, 128, 64] # Important: large to small

# corpus = dataset['corpus']
# queries = dataset['queries']
# relevant_docs = dataset['relevant_docs']

# examples = []
# for query_id, query in queries.items():
#     node_id = relevant_docs[query_id][0]
#     text = corpus[node_id]
#     example = InputExample(texts=[query, text])
#     examples.append(example)

In [6]:
# loader = DataLoader(
#     examples, batch_size=BATCH_SIZE
# )

In [7]:
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [8]:
dataset = val_dataset

corpus = dataset['corpus']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,  # Truncate the embeddings to a certain dimension
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)

# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

first, let's see the accuracy of different dimensions without fine tuning.

In [9]:
# Evaluate the model
results = evaluator(model)

# Print the main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print
    print(f"{key}: {results[key]}")

dim_768_cosine_ndcg@10: 0.43439555076932584
dim_512_cosine_ndcg@10: 0.4261059206595997
dim_256_cosine_ndcg@10: 0.4280086571502763
dim_128_cosine_ndcg@10: 0.4154250006485585
dim_64_cosine_ndcg@10: 0.3755185794895535


In [ ]:
# I also want to compare the openAI embedding performance

model_openai = openai.embedding()

results = evaluator(model_openai)

# Print the main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print
    print(f"{key}: {results[key]}")

Then, we will fine tune the embedding model to see if there is a improvement.

In [10]:
trainset = {
    "queries": [],  # List to store all queries
    "corpus": []    # List to store all corresponding corpus texts
}

# Iterate through the queries and relevant_docs
for query_id, query in train_dataset['queries'].items():
    # Check if the query_id exists in relevant_docs and corpus
    if query_id in train_dataset['relevant_docs']:
        relevant_doc_id = train_dataset['relevant_docs'][query_id][0]  # Get relevant doc ID

        # Check if relevant_doc_id exists in the corpus
        if relevant_doc_id in train_dataset['corpus']:
            # Append query to "queries" list
            trainset['queries'].append(query)

            # Append corresponding corpus text to "corpus" list
            trainset['corpus'].append(train_dataset['corpus'][relevant_doc_id])

In [11]:
from datasets import Dataset

hf_dataset = Dataset.from_dict(trainset)

In [12]:
hf_dataset

Dataset({
    features: ['queries', 'corpus'],
    num_rows: 1090
})

In [17]:
args = SentenceTransformerTrainingArguments(
    output_dir="mpnet-base-v2-matryoshka", # output directory and hugging face model ID
    num_train_epochs=50,                         # number of epochs
    per_device_train_batch_size=32,             # train batch size
    gradient_accumulation_steps=16,             # for a global batch size of 512
    per_device_eval_batch_size=16,              # evaluation batch size
    warmup_ratio=0.1,                           # warmup ratio
    learning_rate=2e-5,                         # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",                 # use constant learning rate scheduler
    optim="adamw_torch_fused",                  # use fused adamw optimizer
    tf32=False,                                  # use tf32 precision
    fp16=True,    #bf16=true                              # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="epoch",                      # evaluate after each epoch
    save_strategy="epoch",                      # save after each epoch
    logging_steps=10,                           # log every 10 steps
    save_total_limit=3,                         # save only the last 3 models
    load_best_model_at_end=True,                # load the best model when training ends
    metric_for_best_model="eval_dim_128_cosine_ndcg@10",  # Optimizing for the best ndcg@10 score for the 128 dimension
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
from sentence_transformers import SentenceTransformerTrainer

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,  # training arguments
    train_dataset=hf_dataset,  # training dataset
    loss=train_loss,
    evaluator=evaluator,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save the best model
trainer.save_model()

Epoch,Training Loss,Validation Loss,Dim 768 Cosine Accuracy@1,Dim 768 Cosine Accuracy@3,Dim 768 Cosine Accuracy@5,Dim 768 Cosine Accuracy@10,Dim 768 Cosine Precision@1,Dim 768 Cosine Precision@3,Dim 768 Cosine Precision@5,Dim 768 Cosine Precision@10,Dim 768 Cosine Recall@1,Dim 768 Cosine Recall@3,Dim 768 Cosine Recall@5,Dim 768 Cosine Recall@10,Dim 768 Cosine Ndcg@10,Dim 768 Cosine Mrr@10,Dim 768 Cosine Map@100,Dim 512 Cosine Accuracy@1,Dim 512 Cosine Accuracy@3,Dim 512 Cosine Accuracy@5,Dim 512 Cosine Accuracy@10,Dim 512 Cosine Precision@1,Dim 512 Cosine Precision@3,Dim 512 Cosine Precision@5,Dim 512 Cosine Precision@10,Dim 512 Cosine Recall@1,Dim 512 Cosine Recall@3,Dim 512 Cosine Recall@5,Dim 512 Cosine Recall@10,Dim 512 Cosine Ndcg@10,Dim 512 Cosine Mrr@10,Dim 512 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Dim 128 Cosine Accuracy@1,Dim 128 Cosine Accuracy@3,Dim 128 Cosine Accuracy@5,Dim 128 Cosine Accuracy@10,Dim 128 Cosine Precision@1,Dim 128 Cosine Precision@3,Dim 128 Cosine Precision@5,Dim 128 Cosine Precision@10,Dim 128 Cosine Recall@1,Dim 128 Cosine Recall@3,Dim 128 Cosine Recall@5,Dim 128 Cosine Recall@10,Dim 128 Cosine Ndcg@10,Dim 128 Cosine Mrr@10,Dim 128 Cosine Map@100,Dim 64 Cosine Accuracy@1,Dim 64 Cosine Accuracy@3,Dim 64 Cosine Accuracy@5,Dim 64 Cosine Accuracy@10,Dim 64 Cosine Precision@1,Dim 64 Cosine Precision@3,Dim 64 Cosine Precision@5,Dim 64 Cosine Precision@10,Dim 64 Cosine Recall@1,Dim 64 Cosine Recall@3,Dim 64 Cosine Recall@5,Dim 64 Cosine Recall@10,Dim 64 Cosine Ndcg@10,Dim 64 Cosine Mrr@10,Dim 64 Cosine Map@100,Sequential Score
0,No log,No log,0.270073,0.507299,0.682482,0.813869,0.270073,0.169100,0.136496,0.081387,0.270073,0.507299,0.682482,0.813869,0.521848,0.430285,0.439398,0.255474,0.514599,0.682482,0.806569,0.255474,0.171533,0.136496,0.080657,0.255474,0.514599,0.682482,0.806569,0.512656,0.420169,0.429177,0.266423,0.532847,0.660584,0.806569,0.266423,0.177616,0.132117,0.080657,0.266423,0.532847,0.660584,0.806569,0.519174,0.428677,0.437855,0.244526,0.496350,0.653285,0.799270,0.244526,0.165450,0.130657,0.079927,0.244526,0.496350,0.653285,0.799270,0.501313,0.407674,0.417356,0.218978,0.445255,0.587591,0.770073,0.218978,0.148418,0.117518,0.077007,0.218978,0.445255,0.587591,0.770073,0.469252,0.375960,0.387683,0.387683
1,No log,No log,0.262774,0.510949,0.678832,0.817518,0.262774,0.170316,0.135766,0.081752,0.262774,0.510949,0.678832,0.817518,0.521053,0.427959,0.436960,0.270073,0.521898,0.675182,0.806569,0.270073,0.173966,0.135036,0.080657,0.270073,0.521898,0.675182,0.806569,0.518817,0.428309,0.437636,0.266423,0.525547,0.667883,0.810219,0.266423,0.175182,0.133577,0.081022,0.266423,0.525547,0.667883,0.810219,0.520750,0.429703,0.439047,0.248175,0.503650,0.656934,0.802920,0.248175,0.167883,0.131387,0.080292,0.248175,0.503650,0.656934,0.802920,0.504814,0.411115,0.420715,0.226277,0.445255,0.598540,0.784672,0.226277,0.148418,0.119708,0.078467,0.226277,0.445255,0.598540,0.784672,0.478493,0.383871,0.394792,0.394792
2,No log,No log,0.270073,0.532847,0.682482,0.824818,0.270073,0.177616,0.136496,0.082482,0.270073,0.532847,0.682482,0.824818,0.527546,0.434162,0.443042,0.270073,0.525547,0.678832,0.821168,0.270073,0.175182,0.135766,0.082117,0.270073,0.525547,0.678832,0.821168,0.524820,0.431856,0.440375,0.273723,0.536496,0.664234,0.817518,0.273723,0.178832,0.132847,0.081752,0.273723,0.536496,0.664234,0.817518,0.527820,0.436721,0.445723,0.251825,0.507299,0.675182,0.806569,0.251825,0.169100,0.135036,0.080657,0.251825,0.507299,0.675182,0.806569,0.507920,0.414021,0.423834,0.229927,0.456204,0.602190,0.806569,0.229927,0.152068,0.120438,0.080657,0.229927,0.456204,0.602190,0.806569,0.4

In [20]:
fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)
# Evaluate the model
results = evaluator(fine_tuned_model)

# # COMMENT IN for full results
# print(results)

# Print the main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print(f"{key}: {results[key]}")

dim_768_cosine_ndcg@10: 0.5962774229140696
dim_512_cosine_ndcg@10: 0.598387254630502
dim_256_cosine_ndcg@10: 0.5941885845319166
dim_128_cosine_ndcg@10: 0.6017626832431484
dim_64_cosine_ndcg@10: 0.5941843312040349


In [ ]:
def get_openai_embedding(text):
    response = openai.Embedding.create(input=text, model="text-embedding-ada-002")
    return response['data'][0]['embedding']